# PART 3 - Machine Learning models
#### Dans cette partie, nous allons diviser nos données, créer des features (OneHot, TF-IDF) et construire nos modèles (Baselines et modèles améliorés)

In [1]:
import pandas as pd 
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import spacy
from urllib.parse import urlparse
import re
import regex
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
# nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from unidecode import unidecode
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from sklearn.compose import ColumnTransformer
import fastparquet

In [2]:
data = Path("../data/").glob("*.parquet") 
data = list(data)

[print(parquet.name) for parquet in data]

part-00000-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00001-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00002-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00003-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00004-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet


[None, None, None, None, None]

In [3]:
df = pd.concat((pd.read_parquet(parquet, engine='fastparquet') for parquet in data))
pd.set_option("max_colwidth", None)


In [4]:
df = df.reset_index(drop=True)
df 

,url,target,day
0,https://www.cdiscount.com/bricolage/electricite/batterie-plomb-6v-4ah-ova51023e-pour-toplux/f-16614-ova2009927775303.html,"[1831, 1751, 1192, 745, 1703]",4
1,https://www.mystalk.net/profile/vitoriafcorrea,"[847, 978, 582, 1381, 529]",4
2,https://www.lequipe.fr/Tennis/TennisFicheJoueur1500000000003017.html,"[20, 1077, 294]",4
3,http://m.jeuxvideo.com/forums/42-32625-60180057-1-0-1-0-la-guilde-fourmi-legionnaire-recrute.htm,"[381, 935, 1343, 622, 933]",4
4,https://context.reverso.net/traduction/espagnol-francais/Para+ir,"[692, 1265, 725, 1264, 1266]",4
...,...,...,...
67590,https://www.jeu-concours.biz/gagner-cafetiere-expresso.html,"[1276, 65, 1113]",1
67591,https://www.sto.cx/book-186042-471.html,"[608, 617, 1033, 220, 1021]",1
67592,http://jeu.info/solution/4-images-1-mot-niveau-2023-a-2060.html,"[381, 925, 622, 1494, 937]",16
67593,https://grossesse.aufeminin.com/forum/levres-gonflees-et-accouchement-fd4242229,"[638, 253, 419, 558, 401]",1


In [5]:
def url_parse(url):
    parse_result = urlparse(url)
    result = [parse_result.scheme, parse_result.netloc, parse_result.path, parse_result.params, parse_result.query, parse_result.fragment]
    return result

df_parsed = pd.concat([df, 
                       pd.DataFrame(list(map(url_parse, df.url)),
                    columns= ['scheme','netloc','path','params','quer','fragment'],
                   index=df.url.index) 
                       ], axis=1)
df_parsed.drop(['params', 'quer', 'fragment'], axis=1, inplace=True)

In [6]:
class PathTokenizer():
    """ A simple class to tokenize the URL with a various combination of functions 
        """
    
    def __init__(self):
        self.stemmer = SnowballStemmer(language='french')
        self.stopwords = [unidecode(x) for x in stopwords.words('french')]
        self.special_words = ['htm', 'php', 'aspx', 'html']

    def _clean_text(self, text:str):
        """ 
        remove the symbols from the a url  
        """
        if isinstance(text, str):
            regex = '(\d+|[A-Z][a-z]*)|[+;,\s.!:\'/_%#&$@?~*]|-'
            t = list(filter(None, re.split(regex, text)))
            return t
        else:
            raise TypeError("text must be list")
    
    def _lowercase_text(self, tokens: list):
        if isinstance(tokens, list):
            return [t.lower() for t in tokens]
        else:
            raise TypeError("text must be list")
    
    def _remove_stopwords(self, tokens:list):
        if isinstance(tokens, list):
            return [t for t in tokens if t not in self.stopwords]
        else:
            raise TypeError("tokens must be a list")
            
    def _remove_single(self, tokens: list):
        "remove single elements from list "
        if isinstance(tokens, list):
            return [t for t in tokens if len(t)>1]
        else:
            raise TypeError("tokens must be a list")
            
    def _remove_specials(self, tokens:list):
        if isinstance(tokens, list):
            return [t for t in tokens if t not in self.special_words]
        else:
            raise TypeError("tokens must be a list")
    
    def _remove_numbers(self, tokens:list):
        if isinstance(tokens, list):
            # return [x for x in text if not any(x1.isdigit() for x1 in x)]
            return [t for t in tokens if not t.isdigit()]
        else:
            raise TypeError("tokens must be a list") 
            
    def _stem_text(self, tokens:list):
        if isinstance(tokens, list):        
            return [self.stemmer.stem(token) for token in tokens]
        else:
            raise TypeError("tokens must be a list")        
        
    def _tokenize_text(self, text:str):
        return word_tokenize(text, language='french')
        
    def _join_words(self, text:list):
        """ build a sentence from a list of words and separates them with a sapce"""
        return " ".join(text)
    
    def _split_words(self, text:str):
        return text.split(' ')
    
    def clean_df(self, df_column, funcs_list):
        "Apply multiple functions on a column of a dataframe"
        for func in funcs_list:
            df_column = df_column.apply(func)
        return df_column

In [7]:
def split_netloc(netloc:str):
    splited_netloc = netloc.rsplit('.', 2)
    if len(splited_netloc) == 2:
        splited_netloc.insert(0, "www")
    return splited_netloc

df_parsed_2 = pd.concat([df_parsed, 
                       pd.DataFrame(list(map(split_netloc, df_parsed.netloc)),
                    columns= ['sous_domaine','domaine','top_domaine'],
                   index=df_parsed.netloc.index) 
                       ], axis=1)

path_tokenizer = PathTokenizer()

funcs = [path_tokenizer._clean_text, path_tokenizer._remove_numbers, path_tokenizer._remove_single, path_tokenizer._stem_text,
        path_tokenizer._lowercase_text, path_tokenizer._remove_specials, path_tokenizer._remove_stopwords ]

df_parsed_2['tokens_path'] = path_tokenizer.clean_df(df_parsed_2.path, funcs)

df_cleaned = df_parsed_2.drop(['url', 'path', 'scheme', 'netloc'], axis=1)

mlb = MultiLabelBinarizer()
targets_encoded = pd.DataFrame(mlb.fit_transform(df_cleaned.target),
                   columns=mlb.classes_,
                   index=df_cleaned.target.index)
df_cleaned_2 = pd.concat([df_cleaned, targets_encoded], axis=1)
df_cleaned_3 = df_cleaned_2.copy(deep=True).drop(columns=['target'])
df_cleaned_3["tokens_path"] = df_cleaned_3.tokens_path.apply(path_tokenizer._join_words)

In [8]:
df_cleaned_3.head(5)

,day,sous_domaine,domaine,top_domaine,tokens_path,100,1000,1001,1002,1003,...,990,991,992,993,994,995,996,997,998,999
0,4,www,cdiscount,com,bricolag electricit batter plomb ah ova toplux ova,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,www,mystalk,net,profil vitoriafcorr,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,www,lequipe,fr,ten ten fich joueur,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,m,jeuxvideo,com,forum guild fourm legionnair recrut,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,context,reverso,net,traduct espagnol franc ir,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split


y = df_cleaned_3.iloc[:, 5:]
x = df_cleaned_3.iloc[:, : 5].drop(['sous_domaine'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

print(X_train.shape)
print(X_test.shape)
X_train

(50696, 4)
(16899, 4)


,day,domaine,top_domaine,tokens_path
4625,11,reverso,net,traduct franc italien nombreux
6279,14,reverso,net,conjugacion franc verbo
62554,9,augsburger-allgemeine,de,donauwoerth eurocopt hebt nach marseil ab id
30717,9,societe,com,dirig xavi
45182,4,cdiscount,com,high tech accessoir android tv box max boiti tv gb gb auc
...,...,...,...,...
29598,9,morgenpost,de,politik articl bundeswehr kaempft gegen borkenkaef politik spricht von katastroph
54058,16,momes,net,comptin comptin march
45062,4,jeuxvideo,com,forum sondag pir arme jeu
11128,16,marmiton,org,recet recet avis brick chevr


## Réduction de la dimension des labels
Nous avons 1903 labels pour notre target ce qui reprensente un nombre assez elevé pour de la classification multi-label.
Nous allons donc effectuer une réduction sur l'espace de dimension des labels.
Plusieurs algorithmes existent pour la réduction de l'espace des labels: compressed sensing (CS) et la  Principal Label Space Transformation (PLST) qui est l'équivalente du PCA sur les features. Egalement des techniques d'embeddings entre targets
Contrairement à ces techniques qui sont indépendentes de nos entrées, il en existe aussi d'autres qui le sont comme par exemple la CPLST ( Conditional Principal Label Space Transformation ) 

Un autre moyen serait d'essayer d'utiliser une approches par embeddings.

Egalement, une autre approche serait d'utiliser 

Pour la classification multi-label, il existe une librarie nommée `scikit-multilearn` qui s'appuie sur la librarie scikit-learn. Elle contient aussi un wrapper autour de  MEKA qui propose une implémentation des méthodes d'apprentissage et d'évaluation multi-labels.

## Construction des features à partir du path
Différentes approches peuvent etre utilisées sur la colonne path:
#### L'utilisation de TF-IDF 
- TF-IDF: refléte l'importance d'un mot pour un document dans une collection (corpus) mais ne prend pas en compte le sens sémantique des mots. TF signifie la probabilité d'occurrence d'un mot dans une phrase.
- TF-IDF donne plus d'importance aux mots qui apparaissent moins fréquemment dans l'ensemble du corpus et donne également de l'importance aux mots les plus fréquents qui apparaissent dans chaque donnée.

Nous testerons les featurizations suivantes:
- TF-IDF: unigrams, bigrams, trigrams et word n-grams.
- TF-IDF based character: unigrams, bigrams, trigrams. (considérer une séquence de caractères plutôt qu'une séquence de mots)


#### L'utilisation des Embeddings (word2vec)
- Word2vec est l'un des modèle de l'état de l'art pour les embeddings.  Il permet de convertir du texte en vecteurs numériques tout en préservantles relations sémantiques entre les mots.
- vecteur de 300 dimensions 

#### L'utilisation d'une combinaison de TF-IDF et moyenne des Embeddings:
Nombiner les n-grammes de caractères avec les vecteurs obtenus avec Word2Vec

In [22]:
labels = df.explode('target')['target'].unique()
labels
len_labels = len(labels)


In [26]:
count_labels = df.explode('target')['target'].value_counts()


In [30]:
len(count_labels[count_labels == 3])

47

In [32]:
count_labels[count_labels == 3]

5006    3
5179    3
5635    3
5486    3
897     3
1251    3
5086    3
5439    3
5811    3
5512    3
5350    3
699     3
1830    3
5867    3
5772    3
1592    3
5627    3
5227    3
1378    3
5450    3
5445    3
5864    3
5026    3
5637    3
1257    3
5169    3
5636    3
5716    3
1575    3
5581    3
1837    3
5507    3
5177    3
5092    3
1666    3
1520    3
1016    3
5131    3
5576    3
1886    3
5568    3
1840    3
1060    3
1744    3
5483    3
5091    3
1678    3
Name: target, dtype: int64

In [37]:
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
X, y = load_linnerud(return_X_y=True)
print(y)

clf = MultiOutputRegressor(Ridge(random_state=123)).fit(X, y)
clf.predict(X[[0]])
print("prdicted",clf.predict(X) )
from numpy import absolute
n_scores = absolute(clf.score(X, y))
print(n_scores)

[[191.  36.  50.]
 [189.  37.  52.]
 [193.  38.  58.]
 [162.  35.  62.]
 [189.  35.  46.]
 [182.  36.  56.]
 [211.  38.  56.]
 [167.  34.  60.]
 [176.  31.  74.]
 [154.  33.  56.]
 [169.  34.  50.]
 [166.  33.  52.]
 [154.  34.  64.]
 [247.  46.  50.]
 [193.  36.  46.]
 [202.  37.  62.]
 [176.  37.  54.]
 [157.  32.  52.]
 [156.  33.  54.]
 [138.  33.  68.]]
prdicted [[176.16484296  35.0548407   57.09000136]
 [188.91063061  37.56282467  54.90134304]
 [189.95342365  37.70903167  53.32601213]
 [183.12565674  35.75756513  55.379517  ]
 [173.71664101  34.19041225  56.86332544]
 [188.24891043  37.14988781  55.05551172]
 [185.98328384  36.4924979   55.17764814]
 [181.88881353  35.85249351  56.12524172]
 [161.29353272  31.59790187  59.28698315]
 [168.78382349  35.14015097  55.24608816]
 [177.5849981   34.49820472  55.9858454 ]
 [167.04250441  33.56470984  57.49566251]
 [164.54944009  32.94682891  58.00146809]
 [201.52146372  39.84126139  52.67320665]
 [193.03036486  37.82070841  54.07885873]


In [54]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import (KNeighborsClassifier,
                               NeighborhoodComponentsAnalysis)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [10]:
from sklearn.base import BaseEstimator,TransformerMixin
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import OneHotEncoder


categorical_features = ["day","domaine","top_domaine"]

tfidf = TfidfVectorizer(min_df=0)
vectorizer = TfidfVectorizer(min_df=0.00009, smooth_idf=True, norm="l2", tokenizer = lambda x: x.split(" "), sublinear_tf=False, ngram_range=(1,1))

transformer = ColumnTransformer([ ('categorical', OneHotEncoder(sparse=False, handle_unknown = "ignore"), categorical_features),
                            ("vectorizer", vectorizer, "tokens_path"),
                         ], remainder="passthrough")

X_train_multilabel = transformer.fit_transform(X_train)
X_test_multilabel = transformer.transform(X_test)
y_train_multilabel = y_train 
y_test_multilabel = y_test 

numpy.int32

In [70]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(y_train)
print(svd.explained_variance_ratio_.sum())



0.7795380660263765


In [74]:
y_train_pca = svd.transform(y_train)
y_test_pca = svd.transform(y_test)
print(y_train_pca)
print(y_test_pca)

[[ 1.16698157e-03  2.02205174e-01 -1.51674001e-03 ... -9.06461679e-02
  -7.46545276e-02  1.07088520e-01]
 [ 3.29729426e-04  5.68230880e-04  2.30665080e-03 ...  6.64352325e-02
   5.20180074e-02  2.03716402e-02]
 [ 8.08941869e-04  4.15118428e-03  1.65159303e-02 ... -1.73589724e-02
   7.71876700e-02  6.36869989e-02]
 ...
 [ 1.71505005e-04  2.94716032e-04  9.24831989e-04 ... -8.14931007e-02
   4.41209369e-02 -9.26041634e-02]
 [ 1.32073693e-03  7.05162267e-04  3.08925341e-03 ...  7.68610501e-04
   9.07694964e-03 -9.02635115e-03]
 [ 5.26857327e-04  2.36930098e-03  3.84579757e-03 ...  5.97734134e-02
   2.39768145e-02 -1.23545101e-02]]
[[ 6.43451589e-04  2.24598175e-04  3.19885197e-04 ...  4.87054545e-03
  -1.48145589e-02  1.23834718e-02]
 [ 1.00996203e-03  4.89659300e-03  3.56150823e-03 ... -6.84465562e-02
   8.60469400e-03  1.06722370e-01]
 [ 1.78608895e-03  3.01563255e-03  3.03268881e-03 ...  4.63070562e-03
   7.07963553e-03  8.13964791e-03]
 ...
 [ 3.67130314e-04  8.79875142e-04  2.4779079

In [ ]:
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
# define dataset

# define base model
model = LinearSVR()
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(wrapper, X_train_multilabel, y_train_pca, scoring='neg_mean_absolute_error', n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [59]:
n_neighbors = 3
random_state = 0


dim = len_labels
n_classes = len_labels

# Reduce dimension to 100 with PCA
pca = make_pipeline(StandardScaler(),
                    PCA(n_components=100, random_state=random_state))

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


pca = PCA(n_components=2)
X_r = pca.fit(X).transform(y_train)

lda = LinearDiscriminantAnalysis(n_components=2)
X_r2 = lda.fit(X, y).transform(X)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))

plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of IRIS dataset')

plt.figure()
for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r2[y == i, 0], X_r2[y == i, 1], alpha=.8, color=color,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA of IRIS dataset')

plt.show()

ValueError: Pipeline.fit does not accept the with_mean parameter. You can pass parameters to specific steps of your pipeline using the stepname__parameter format, e.g. `Pipeline.fit(X, y, logisticregression__sample_weight=sample_weight)`.

<Figure size 432x288 with 0 Axes>

In [ ]:
print(pca_model.explained_varianceratio.cumsum())

## Construction des features à partir du path
Différentes approches peuvent etre utilisées sur la colonne path:
#### L'utilisation de TF-IDF 
- TF-IDF: refléte l'importance d'un mot pour un document dans une collection (corpus) mais ne prend pas en compte le sens sémantique des mots. TF signifie la probabilité d'occurrence d'un mot dans une phrase.
- TF-IDF donne plus d'importance aux mots qui apparaissent moins fréquemment dans l'ensemble du corpus et donne également de l'importance aux mots les plus fréquents qui apparaissent dans chaque donnée.

Nous testerons les featurizations suivantes:
- TF-IDF: unigrams, bigrams, trigrams et word n-grams.
- TF-IDF based character: unigrams, bigrams, trigrams. (considérer une séquence de caractères plutôt qu'une séquence de mots)


#### L'utilisation des Embeddings (word2vec)
- Word2vec est l'un des modèle de l'état de l'art pour les embeddings.  Il permet de convertir du texte en vecteurs numériques tout en préservantles relations sémantiques entre les mots.
- vecteur de 300 dimensions 

#### L'utilisation d'une combinaison de TF-IDF et moyenne des Embeddings:
Nombiner les n-grammes de caractères avec les vecteurs obtenus avec Word2Vec

### Création des modèles :
#### Baseslines: 
- Dans un premier temps, nous allons construire nos modèles Baselines avec l'ensemble des targets présentes dans notre dataset. Chaque modèle essaiera de prédire les targets sur le nombre total de 1009 targets. 
- Après avoir construit les baslines models, nous essaierons d'améliorer les modèles avec des hyperparamètres pour voir s'il y a une augmentation du score F1.

#### Aller plus loin
- Ensuite, nous construirons nos modèles en séléctionnant peu de colonnes par exemple.


Logistic Regression, Linear SVMs, SGD Classifiers avec log loss,
SGD Classifier avec hinge loss
objectif : maximiser le micro averaged F1 score 

### Multi-label classification
#### 1.  OneVsRest: 
Le problème est décomposé en un problème de classification binaire multiple. Nous choisissons une classe et formons un classificateur binaire avec les échantillons de la classe sélectionnée d'un côté et tous les autres échantillons de l'autre côté. Ainsi, nous obtiendrons N classificateurs pour N étiquettes et lors du test nous classerons simplement l'échantillon comme appartenant à la classe avec le score maximum parmi les N classificateurs.

In [11]:
classifier1 = OneVsRestClassifier(LogisticRegression(penalty='l1', solver='liblinear', class_weight="balanced"), n_jobs=-1)
classifier1.fit(X_train_multilabel, y_train_multilabel)
predictions = classifier1.predict(X_test_multilabel)

KeyboardInterrupt: 

In [37]:
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score

print("Accuracy :",metrics.accuracy_score(y_test_multilabel, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test_multilabel,predictions))
precision = precision_score(y_test_multilabel, predictions, average='micro')
recall = recall_score(y_test_multilabel, predictions, average='micro')
f1 = f1_score(y_test_multilabel, predictions, average='micro')
 
print("\nMicro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
precision = precision_score(y_test_multilabel, predictions, average='macro')
recall = recall_score(y_test_multilabel, predictions, average='macro')
f1 = f1_score(y_test_multilabel, predictions, average='macro')
 
print("\nMacro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
print("\nClassification Report")
print (metrics.classification_report(y_test_multilabel, predictions))

Accuracy : 0.007278537191549796
Hamming loss  0.0068070643314176215

Micro-average quality numbers
Precision: 0.2285, Recall: 0.7251, F1-measure: 0.3475

Macro-average quality numbers
Precision: 0.1337, Recall: 0.3929, F1-measure: 0.1884

Classification Report
              precision    recall  f1-score   support

           0       0.05      0.35      0.08        23
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         1
           3       0.10      0.20      0.13         5
           4       0.20      0.70      0.31        43
           5       0.09      0.33      0.14        18
           6       0.00      0.00      0.00         0
           7       0.07      0.23      0.10        22
           8       0.06      0.15      0.09        13
           9       0.03      0.25      0.06         4
          10       0.33      0.20      0.25         5
          11       0.12      0.54      0.19        52
          12       0.13      0.36   

In [45]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer(binary='true', max_features=3)
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

['is', 'the', 'this']


#### 2.Binary Relevance:
Transformer un problème multi-label avec K étiquettes en des problèmes de classification binaire séparés. Chaque classificateur prédit si une étiquette est présente ou non.

Les deux techniques présentées en haut traitent le problème multi-label (choix multiples) en une série de questions oui/non, mais la dernière traite des choix mutuellement exclusifs.

#### 3.Classifier Chains:
Un classificateur 1 sera formé sur les données d'entrée. La sortie du classificateur 1 sera alimentée en entrée pour le classificateur 2, qui prédit la deuxième étiquette, la sortie du classificateur 2 sera alimentée en entrée pour le classificateur 3 et ainsi de suite.

In [11]:
import joblib
import sys


sys.modules['sklearn.externals.joblib'] = joblib

from skmultilearn.embedding import OpenNetworkEmbedder
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder


graph_builder = LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False)
openne_line_params = dict(batch_size=1000, order=3)
embedder = OpenNetworkEmbedder(
    graph_builder,
    'LINE',
    dimension = 5*df_cleaned_3.iloc[:, 5:].values.shape[1],
    aggregation_function = 'add',
    normalize_weights=True,
    param_dict = openne_line_params
)
from skmultilearn.embedding import EmbeddingClassifier
from sklearn.ensemble import RandomForestRegressor
from skmultilearn.adapt import MLkNN


clf = EmbeddingClassifier(
    embedder,
    RandomForestRegressor(n_estimators=10),
    MLkNN(k=5)
)

In [12]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

clf.fit(X_train_multilabel, y_train.values)

Pre-procesing for non-uniform negative sampling!


AttributeError: module 'tensorflow' has no attribute 'contrib'

In [ ]:
predictions = clf.predict(X_test_multilabel)

#### Trois approches peuvent etre prises pour la colonne sous_domaine:
- La prétraiter (enlever les www et les lettres, tokenizer...) et l'ajouter à la colonne path pour former une description
- L'enlever 
- La prétraiter afin de construire une feature catégorielle.
- Construire directement une feature catégorielle sans prétraitement

Pour l'instant nous optons pour la 2eme approche